In [217]:
from gurobipy import GRB
import gurobipy as gb


# QUESTION 1

## Extracting the necessary data

In [218]:
import pandas as pd

# Load data files
file_paths = {
    "direct_capacity": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Direct_Production_Facilities.csv",
    "transship_capacity_centers": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Distribution_Centers.csv",
    "transship_capacity_facilities": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Capacity_for_Transship_Production_Facilities.csv",
    "cost_production_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Refinement.csv",
    "cost_production_transshipment": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Production_to_Transshipment.csv",
    "cost_transshipment_refinement": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Cost_Transshipment_to_Refinement.csv",
    "refinement_demand": "https://raw.githubusercontent.com/annwanginnt/Operation-Research/main/Refinement_Demand.csv",
}

# Reading the data
data = {key: pd.read_csv(file_paths[key]) for key in file_paths}

# Displaying the keys to understand the structure of the data
data.keys()  


dict_keys(['direct_capacity', 'transship_capacity_centers', 'transship_capacity_facilities', 'cost_production_refinement', 'cost_production_transshipment', 'cost_transshipment_refinement', 'refinement_demand'])

In [219]:
# Displaying the first few rows of each dataset to understand their structure
data_overview = {key: data[key].head() for key in data}

data_overview


{'direct_capacity':    ProductionFacility  Capacity
 0                   1       462
 1                   2       103
 2                   3       460
 3                   4       325
 4                   5       227,
 'transship_capacity_centers':    TransshipmentHub  Capacity
 0                 1      1317
 1                 2      1453,
 'transship_capacity_facilities':    ProductionFacility  Capacity
 0                   1       374
 1                   2       444
 2                   3       395
 3                   4       245
 4                   5       378,
 'cost_production_refinement':    ProductionFacility  RefinementCenter      Cost
 0                   1                 1  4.252733
 1                   1                 2  4.567726
 2                   1                 3  4.696484
 3                   1                 4  2.678741
 4                   1                 5  4.272451,
 'cost_production_transshipment':    ProductionFacility  TransshipmentHub      Cost
 0   

In [220]:
data['refinement_demand']

,RefinementCenter,Demand
0,1,1537
1,2,1748
2,3,1940
3,4,1838
4,5,1665


In [221]:
# Extracting the necessary data

# Capacities array
cap_direct = data['direct_capacity']['Capacity'].values  
cap_transship_facilities = data['transship_capacity_facilities']['Capacity'].values
cap_transship_centers = data['transship_capacity_centers']['Capacity'].values

# Costs dataframe
cost_prod_ref = data['cost_production_refinement']
cost_prod_trans = data['cost_production_transshipment']
cost_trans_ref = data['cost_transshipment_refinement']



# Number of facilities and centers
num_direct_facilities = len(cap_direct)
num_transship_facilities = len(cap_transship_facilities)
num_refinement_centers = 5  # Given in the problem statement
num_transship_centers = len(cap_transship_centers)

# demand 
demand_data = data["refinement_demand"]

#total demand
total_demand = demand_data['Demand'].sum()

print('Capacity of direct facilities:', cap_direct)
print('Capacity of transship facilities:', cap_transship_facilities)
print('Capacity of distribution center:',cap_transship_centers)


print('\n')


print('number of direct facility:', num_direct_facilities)
print('number of transship facility:', num_transship_facilities)
print('number of distirbution center', num_transship_centers)
print('number of refinement center:',num_refinement_centers)

print('\n')

print('Demand:', demand_data)

print('\n')

print('total_demand:',total_demand)



Capacity of direct facilities: [462 103 460 325 227 217 205 521 548 191 361 411 104 155 285 109 422 438
 501 139 462 504 106 132 298]
Capacity of transship facilities: [374 444 395 245 378 408 435 175 415 503 184 297 450 169 365]
Capacity of distribution center: [1317 1453]


number of direct facility: 25
number of transship facility: 15
number of distirbution center 2
number of refinement center: 5


Demand:    RefinementCenter  Demand
0                 1    1537
1                 2    1748
2                 3    1940
3                 4    1838
4                 5    1665


total_demand: 8728


## Create the optimization model

In [222]:
from gurobipy import GRB, Model

# Create the optimization model
model_a = Model("Transshipment Problem")


## Define decision variables

In [223]:
# Define the indices for the facilities and centers
direct_facilities = range(num_direct_facilities)  
transship_facilities = range(num_transship_facilities)  
refinement_centers = range(num_refinement_centers)  
transship_centers = range(num_transship_centers)  

# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model.addVars(direct_facilities, refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model.addVars(transship_facilities, transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model.addVars(transship_centers, refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")

## Define objective function

In [224]:
#使得我们能够准确地从一个结构化的数据表中提取出对应每条运输路线的成本数据。
# Objective function: Minimize total cost
model.setObjective(gb.quicksum(cost_prod_ref.iloc[i * num_refinement_centers + j]['Cost'] * X[i, j] 
                               for i in range(num_direct_facilities) 
                               for j in range(num_refinement_centers)) +
                   gb.quicksum(cost_prod_trans.iloc[i * num_transship_centers + j]['Cost'] * Y[i, j] 
                               for i in range(num_transship_facilities) 
                               for j in range(num_transship_centers)) +
                   gb.quicksum(cost_trans_ref.iloc[j * num_refinement_centers + k]['Cost'] * Z[j, k] 
                               for j in range(num_transship_centers) 
                               for k in range(num_refinement_centers)),
                   GRB.MINIMIZE)




## Define constraint

In [225]:
# Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)


## Optimize model

In [226]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 57 rows, 178 columns and 326 nonzeros
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+05]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.728000e+03   0.000000e+00      0s
       6    1.7267260e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.726726025e+04


## Question(a): After solving the linear program, what is the optimal transportation cost?

In [227]:
# Check if the solution is found
if model.status == GRB.Status.OPTIMAL:
    optimal_cost_a = model.objVal
    print(f"Optimal Transportation Cost_a: {optimal_cost_a}")
else:
    print("No optimal solution found")

Optimal Transportation Cost_a: 17267.26024720385


## Question(b): In the optimal solution, what proportion of canola oil is transshipped?

In [228]:
# 计算总的转运量
total_transshipped = sum(Y[i, j].X for i in transship_facilities for j in transship_centers) + \
                     sum(Z[j, k].X for j in transship_centers for k in refinement_centers)

# 计算总的运输量
total_transport = total_transshipped + \
                  sum(X[i, j].X for i in direct_facilities for j in refinement_centers)

# 计算比例
proportion_transshipped = total_transshipped / total_transport if total_transport > 0 else 0

print('% of transshipped:',proportion_transshipped)


% of transshipped: 0.9470669110907425



 ## Question(c): The model does not currently limit that amount of canola oil that is transshipped. How would you modify the objective function to account for this? Formulate and solve this model.

In [229]:
# Create the optimization model
model = Model("modify the objective function")


# define tranship_cost_per_unit

#avg_cost_direct = cost_prod_ref['Cost'].mean()
avg_cost_transship = (cost_prod_trans['Cost'] + cost_trans_ref['Cost']).mean()
#transship_cost_per_unit = avg_cost_transship - avg_cost_direct
#print('tranship_cost_per_unit:',transship_cost_per_unit)

transship_cost_per_unit =2

print('\n')

# Define the indices for the facilities and centers
direct_facilities = range(num_direct_facilities)  
transship_facilities = range(num_transship_facilities)  
refinement_centers = range(num_refinement_centers)  
transship_centers = range(num_transship_centers)  

# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model.addVars(direct_facilities, refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model.addVars(transship_facilities, transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model.addVars(transship_centers, refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")



# Add tranship_cost_per_unit in objective function
model.setObjective(gb.quicksum(cost_prod_ref.iloc[i * num_refinement_centers + j]['Cost'] * X[i, j] 
                               for i in range(num_direct_facilities) 
                               for j in range(num_refinement_centers)) +
                   gb.quicksum(cost_prod_trans.iloc[i * num_transship_centers + j]['Cost'] * Y[i, j] 
                               for i in range(num_transship_facilities) 
                               for j in range(num_transship_centers)) +
                   gb.quicksum(cost_trans_ref.iloc[j * num_refinement_centers + k]['Cost'] * Z[j, k] 
                               for j in range(num_transship_centers) 
                               for k in range(num_refinement_centers)) +
                   transship_cost_per_unit * (sum(Y[i, j] for i in transship_facilities for j in transship_centers) + 
                                              sum(Z[j, k] for j in transship_centers for k in refinement_centers)),
                   GRB.MINIMIZE)


# Define Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)


model.optimize()

    
if model.status == GRB.Status.OPTIMAL:
    optimal_cost_c = model.objVal
    print(f"Optimal Transportation Cost_c: {optimal_cost_c}")
else:
    print("No optimal solution found")



Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 45 rows, 165 columns and 290 nonzeros
Model fingerprint: 0xaad894fe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 30 rows and 122 columns
Presolve time: 0.01s
Presolved: 15 rows, 43 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5060126e+03   8.725000e+02   0.000000e+00      0s
      16    2.7511645e+04   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.751164520e+04
Optimal Transportation Cost_c: 27511.645199616018


## Quesion(d) Instead of modifying the objective function, how would you modify the constraint set to reduce the proportion of canola oil that is transshipped? Formulate and solve this model.

In [230]:
model = Model("Can2Oil Transshipment Problem")


# 计算所有转运中心的总容量
cap_transship_centers = data['transship_capacity_centers']['Capacity'].values
total_transship_center_capacity = sum(cap_transship_centers)


# 将转运量的上限设定为转运中心能力的 50%
#max_transship_amount = 0.5 * total_transship_center_capacity
max_transship_amount = 0.5* total_demand


# Define the indices for the facilities and centers
direct_facilities = range(num_direct_facilities)  
transship_facilities = range(num_transship_facilities)  
refinement_centers = range(num_refinement_centers)  
transship_centers = range(num_transship_centers)  

# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model.addVars(direct_facilities, refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model.addVars(transship_facilities, transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model.addVars(transship_centers, refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")


# define Objective function: Minimize total cost
model.setObjective(gb.quicksum(cost_prod_ref.iloc[i * num_refinement_centers + j]['Cost'] * X[i, j] 
                               for i in range(num_direct_facilities) 
                               for j in range(num_refinement_centers)) +
                   gb.quicksum(cost_prod_trans.iloc[i * num_transship_centers + j]['Cost'] * Y[i, j] 
                               for i in range(num_transship_facilities) 
                               for j in range(num_transship_centers)) +
                   gb.quicksum(cost_trans_ref.iloc[j * num_refinement_centers + k]['Cost'] * Z[j, k] 
                               for j in range(num_transship_centers) 
                               for k in range(num_refinement_centers)),
                   GRB.MINIMIZE)



# add Constraints
model.addConstr(sum(Y[i, j] for i in transship_facilities for j in transship_centers) +
                sum(Z[j, k] for j in transship_centers for k in refinement_centers) <= max_transship_amount)

# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)

# optimize the model    
model.optimize()

print('\n')
# Check if the solution is found
if model.status == GRB.Status.OPTIMAL:
    optimal_cost_d = model.objVal
    print(f"Optimal Transportation Cost_d: {optimal_cost_d}")
else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 46 rows, 165 columns and 330 nonzeros
Model fingerprint: 0xe972b87e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 4e+03]
Presolve removed 15 rows and 35 columns
Presolve time: 0.01s
Presolved: 31 rows, 130 columns, 260 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7230583e+04   6.596921e+02   0.000000e+00      0s
      23    1.9316094e+04   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.931609402e+04


Optimal Transportation Cost_d: 19316.09402337471


## Question (e) Which of the two modeling approaches would you recommend the company take to determine a transportation plan that reduces the amount of canola oil that is transshipped?

The approach in question(d) would be recommended: modify the constraint set to reduce the proportion of canola oil that is transshipped

## Question(f): re-shoring is the practice of transferring overseas business operations closer to the home country. Given its prevalence in today’s economy, how would you alter the original model to favor producers closer to North America? Formulate and solve this model.


In [231]:
import pandas as pd
from gurobipy import GRB, Model
import gurobipy as gb

# create model
model = Model("Can2Oil Transshipment Problem")


# Define the indices for the facilities and centers
direct_facilities = range(num_direct_facilities)  
transship_facilities = range(num_transship_facilities)  
refinement_centers = range(num_refinement_centers)  
transship_centers = range(num_transship_centers)  

# Create decision variables
# Xij - Quantity from direct production facility i to refinement center j
X = model.addVars(direct_facilities, refinement_centers,lb =0, vtype=GRB.CONTINUOUS, name="X")

# Yij - Quantity from transshipment production facility i to transshipment center j
Y = model.addVars(transship_facilities, transship_centers,lb=0, vtype=GRB.CONTINUOUS, name="Y")

# Zjk - Quantity from transshipment center j to refinement center k
Z = model.addVars(transship_centers, refinement_centers,lb=0, vtype=GRB.CONTINUOUS, name="Z")

# define Objective function: Minimize total cost
model.setObjective(gb.quicksum(cost_prod_ref.iloc[i * num_refinement_centers + j]['Cost'] * X[i, j] 
                               for i in range(num_direct_facilities) 
                               for j in range(num_refinement_centers)) +
                   gb.quicksum(cost_prod_trans.iloc[i * num_transship_centers + j]['Cost'] * Y[i, j] 
                               for i in range(num_transship_facilities) 
                               for j in range(num_transship_centers)) +
                   gb.quicksum(cost_trans_ref.iloc[j * num_refinement_centers + k]['Cost'] * Z[j, k] 
                               for j in range(num_transship_centers) 
                               for k in range(num_refinement_centers)),
                   GRB.MINIMIZE)

# Constraints
# Capacity constraints for direct production facilities
for i in range(num_direct_facilities):
    model.addConstr(gb.quicksum(X[i, j] for j in range(num_refinement_centers)) <= cap_direct[i])

# Capacity constraints for transshipment production facilities
for i in range(num_transship_facilities):
    model.addConstr(gb.quicksum(Y[i, j] for j in range(num_transship_centers)) <= cap_transship_facilities[i])

# Demand constraints at each refinement center
for j in range(num_refinement_centers):
    demand_j = demand_data.loc[j, 'Demand']  
    model.addConstr(gb.quicksum(X[i, j] for i in range(num_direct_facilities)) +
                    gb.quicksum(Z[k, j] for k in range(num_transship_centers)) == demand_j)



# 确定更接近北美的生产设施
north_america_facilities = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]  # 示例编号

# 调整这些设施的运输成本
cost_reduction_factor = 0.9  # 例如减少 10%
cost_prod_ref.loc[cost_prod_ref['ProductionFacility'].isin(north_america_facilities), 'Cost'] *= cost_reduction_factor


# 求解模型
model.optimize()

# 输出结果
if model.status == GRB.Status.OPTIMAL:
    optimal_cost_f = model.objVal
    print(f"Optimal Transportation Cost_f: {optimal_cost_f}")
else:
    print("No optimal solution found")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 45 rows, 165 columns and 290 nonzeros
Model fingerprint: 0x97134bb6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 2e+03]
Presolve removed 45 rows and 165 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7267260e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.726726025e+04
Optimal Transportation Cost_f: 17267.26024720385


## Question (g) Do you expect the optimal solution to the re-shoring model to be similar to the optimal solution of the model that attempts to reduce transshipment? Why or why not?

Answer : Yes, re-shoring model would be better solution due to the total transportion cost with re-shoring solution is lower than the solution with reducing the transshipment. 

In [232]:
# create a dictionalyr
results = {
    "Model A": optimal_cost_a,
    "Model C": optimal_cost_c,
    "Model D": optimal_cost_d,
    "Model F": optimal_cost_f
}

# convert to dataframe
results_df = pd.DataFrame(list(results.items()), columns=["Model", "Optimal Cost"])

sorted_results_df = results_df.sort_values(by="Optimal Cost")

print(sorted_results_df)

best_model = sorted_results_df.iloc[0]
print(f"The best solution is provided by {best_model['Model']} with a cost of {best_model['Optimal Cost']}")

     Model  Optimal Cost
0  Model A  17267.260247
3  Model F  17267.260247
2  Model D  19316.094023
1  Model C  27511.645200
The best solution is provided by Model A with a cost of 17267.26024720385


# Question 2

In [233]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("SunnyshoreBay")

# Time periods
months = ["May", "June", "July"]

# Revenues and Expenses for each month
revenues = {"May": 180000, "June": 260000, "July": 420000, "August": 580000}
expenses = {"May": 300000, "June": 400000, "July": 350000, "August": 200000}

# Initial cash balance
initial_balance = 140000

# Borrowing limits for each month
borrowing_limits = {"May": 250000, "June": 150000, "July": 350000}

# Interest rates for different term structures
interest_rates = {1: 1.0175, 2: 1.0225, 3: 1.0275}

# Minimum cash balance requirements
min_cash_balance = {"May": 25000, "June": 20000, "July": 35000, "August": 18000}

# Add decision variables for borrowing amounts
borrowing_vars = {}
for month in months:
    for term in interest_rates:
        borrowing_vars[(month, term)] = model.addVar(vtype=GRB.CONTINUOUS, name=f"Borrow_{month}_{term}m")

# Cash balance variables for each month
cash_balance_vars = {}
for month in ["May", "June", "July", "August"]:
    cash_balance_vars[month] = model.addVar(vtype=GRB.CONTINUOUS, name=f"CashBalance_{month}")

# Objective: Minimize total repayment
total_repayment = gp.quicksum(borrowing_vars[month, term] * interest_rates[term] 
                              for month in months for term in interest_rates)
model.setObjective(total_repayment, GRB.MINIMIZE)

# Constraint: Cash balance at the end of each month
# May
model.addConstr(cash_balance_vars["May"] == initial_balance + revenues["May"] - expenses["May"] 
                + gp.quicksum(borrowing_vars["May", term] for term in interest_rates), 
                "CashBalance_May")

# June
model.addConstr(cash_balance_vars["June"] == cash_balance_vars["May"] + revenues["June"] - expenses["June"] 
                + gp.quicksum(borrowing_vars["June", term] for term in interest_rates) 
                - borrowing_vars["May", 1], "CashBalance_June")

# July
model.addConstr(cash_balance_vars["July"] == cash_balance_vars["June"] + revenues["July"] - expenses["July"] 
                + gp.quicksum(borrowing_vars["July", term] for term in interest_rates if term != 3) 
                - borrowing_vars["June", 1] - borrowing_vars["May", 2], "CashBalance_July")

# August
model.addConstr(cash_balance_vars["August"] == cash_balance_vars["July"] + revenues["August"] - expenses["August"] 
                - borrowing_vars["July", 1] - borrowing_vars["June", 2] - borrowing_vars["May", 3], 
                "CashBalance_August")

# Constraint: Borrowing limits
for month in months:
    model.addConstr(gp.quicksum(borrowing_vars[month, term] for term in interest_rates if month != "July" or term != 3) 
                    <= borrowing_limits[month], f"BorrowingLimit_{month}")

# Constraint: Minimum cash balance requirements
for month in ["May", "June", "July", "August"]:
    model.addConstr(cash_balance_vars[month] >= min_cash_balance[month], f"MinCash_{month}")

# Constraint: July cash balance requirement
model.addConstr(cash_balance_vars["July"] >= 0.65 * (cash_balance_vars["May"] + cash_balance_vars["June"]), "JulyCashRequirement")

# Optimize model
model.optimize()

# Retrieve and print the optimal solutions
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for month in ["May", "June", "July", "August"]:
        print(f"Cash balance at the end of {month}: ${cash_balance_vars[month].X:.2f}")
    for month in months:
        for term in interest_rates:
            print(f"Amount borrowed at the end of {month} for a {term}-month term: ${borrowing_vars[(month, term)].X:.2f}")

    # Print total repayment amount
    print(f"Total repayment amount: ${model.objVal:.2f}")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 13 columns and 36 nonzeros
Model fingerprint: 0x3ce3d8f1
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 8 rows, 11 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       5    1.4290000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429000000e+05
Optimal solution found:
Cash balance at the end of May: $25000.00
Cash balance at the end of June: $20000.00
Cash balan

## question (a): how many different investment can be made over the 4-month period?

In [234]:
# Borrowing options in each month
options_may = 4 # 3 borrowing options (1-month, 2-months, 3-months) + 1 option of not borrowing
options_june = 3 # 2 borrowing options (1-month, 2-months) + 1 option of not borrowing
options_july = 2 # 1 borrowing option (1-month) + 1 option of not borrowing

# Total number of different investments
total_investments = options_may * options_june * options_july
print('total_investments: ',total_investments)


total_investments:  24


## question(b): Write down the cash balance constraint for money on-hand at the end of June

June cash balance :

model.addConstr(cash_balance_vars["June"] == cash_balance_vars["May"] + revenues["June"] - expenses["June"] 
                + gp.quicksum(borrowing_vars["June", term] for term in interest_rates) 
                - borrowing_vars["May", 1], "CashBalance_June")

In [235]:

# Initialize Gurobi model
model = gp.Model("SunnyshoreBay_JuneCashBalance")

# Data 
initial_cash_balance = 140000  # Initial cash balance
revenues = {"May": 180000, "June": 260000}  # Revenues for May and June
expenses = {"May": 300000, "June": 400000}  # Expenses for May and June
min_cash_june = 20000  # Minimum required cash balance for June

# Decision Variables
borrow_may = model.addVar(vtype=GRB.CONTINUOUS, name="borrow_may")
borrow_june = model.addVar(vtype=GRB.CONTINUOUS, name="borrow_june")
repay_may = model.addVar(vtype=GRB.CONTINUOUS, name="repay_may")  # Repayment for 1-month term loan taken in May
cash_june = model.addVar(vtype=GRB.CONTINUOUS, name="cash_june")  # Cash balance at the end of June

# Constraint: Equation for Cash balance at the end of June
model.addConstr(cash_june == initial_cash_balance + revenues["May"] - expenses["May"] + revenues["June"] - expenses["June"] + borrow_may + borrow_june - repay_may, "JuneCashBalance")

# Constraint: Cash balance at the end of June should be at least the minimum required
model.addConstr(cash_june >= min_cash_june, "MinCashBalanceJune")

# Solve the model (this is just for illustration, normally you would have other constraints and objective)
model.optimize()

# Print solution (if available)
if model.status == GRB.OPTIMAL:
    print(f"Cash balance at the end of June: ${cash_june.X:.2f}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 2 rows, 4 columns and 5 nonzeros
Model fingerprint: 0x17db0ebe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 1e+05]
Presolve removed 2 rows and 4 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Cash balance at the end of June: $20000.00


## question(c): Write down the linear ratio constraint associated with the cash balance at the end of July.


model.addConstr(cash_balance_vars["July"] >= 0.65 * (cash_balance_vars["May"] + cash_balance_vars["June"]), "JulyCashRequirement")


## quesiton(d): What is the total amount that Sunnyshore Bay has to repay to the bank over the entire season

Total repayment = sum( principle + principle * interest rate)

total_repayment = gp.quicksum(borrowing_vars[month, term] * interest_rates[term] 
                              for month in months for term in interest_rates)

Total repayment amount: $142900.00


## question(e): How much money does Sunnyshore Bay withdraw in May from all loans?

Amount borrowed at the end of May for a 1-month term: $0.00
Amount borrowed at the end of May for a 2-month term: $5000.00
Amount borrowed at the end of May for a 3-month term: $0.00

so totally it is $5000

## question(f): What is the cash balance at the end of August?

Cash balance at the end of August: $330000.00

## question(g): Due to potential unexpected repairs, one of the managers has suggested increasing the minimum cash balance for June to $27,500. How much will now have to be repaid if this change is approved?

Total repayment amount will be increase as  $150531.25  from original Total repayment amount: $142900.00

In [236]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("SunnyshoreBay")

# Time periods
months = ["May", "June", "July"]

# Revenues and Expenses for each month
revenues = {"May": 180000, "June": 260000, "July": 420000, "August": 580000}
expenses = {"May": 300000, "June": 400000, "July": 350000, "August": 200000}

# Initial cash balance
initial_balance = 140000

# Borrowing limits for each month
borrowing_limits = {"May": 250000, "June": 150000, "July": 350000}

# Interest rates for different term structures
interest_rates = {1: 1.0175, 2: 1.0225, 3: 1.0275}

# Minimum cash balance requirements
min_cash_balance = {"May": 25000, "June": 27500, "July": 35000, "August": 18000}

# Add decision variables for borrowing amounts
borrowing_vars = {}
for month in months:
    for term in interest_rates:
        borrowing_vars[(month, term)] = model.addVar(vtype=GRB.CONTINUOUS, name=f"Borrow_{month}_{term}m")

# Cash balance variables for each month
cash_balance_vars = {}
for month in ["May", "June", "July", "August"]:
    cash_balance_vars[month] = model.addVar(vtype=GRB.CONTINUOUS, name=f"CashBalance_{month}")

# Objective: Minimize total repayment
total_repayment = gp.quicksum(borrowing_vars[month, term] * interest_rates[term] 
                              for month in months for term in interest_rates)
model.setObjective(total_repayment, GRB.MINIMIZE)

# Constraint: Cash balance at the end of each month
# May
model.addConstr(cash_balance_vars["May"] == initial_balance + revenues["May"] - expenses["May"] 
                + gp.quicksum(borrowing_vars["May", term] for term in interest_rates), 
                "CashBalance_May")

# June
model.addConstr(cash_balance_vars["June"] == cash_balance_vars["May"] + revenues["June"] - expenses["June"] 
                + gp.quicksum(borrowing_vars["June", term] for term in interest_rates) 
                - borrowing_vars["May", 1], "CashBalance_June")

# July
model.addConstr(cash_balance_vars["July"] == cash_balance_vars["June"] + revenues["July"] - expenses["July"] 
                + gp.quicksum(borrowing_vars["July", term] for term in interest_rates if term != 3) 
                - borrowing_vars["June", 1] - borrowing_vars["May", 2], "CashBalance_July")

# August
model.addConstr(cash_balance_vars["August"] == cash_balance_vars["July"] + revenues["August"] - expenses["August"] 
                - borrowing_vars["July", 1] - borrowing_vars["June", 2] - borrowing_vars["May", 3], 
                "CashBalance_August")

# Constraint: Borrowing limits
for month in months:
    model.addConstr(gp.quicksum(borrowing_vars[month, term] for term in interest_rates if month != "July" or term != 3) 
                    <= borrowing_limits[month], f"BorrowingLimit_{month}")

# Constraint: Minimum cash balance requirements
for month in ["May", "June", "July", "August"]:
    model.addConstr(cash_balance_vars[month] >= min_cash_balance[month], f"MinCash_{month}")

# Constraint: July cash balance requirement
model.addConstr(cash_balance_vars["July"] >= 0.65 * (cash_balance_vars["May"] + cash_balance_vars["June"]), "JulyCashRequirement")

# Optimize model
model.optimize()

# Retrieve and print the optimal solutions
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for month in ["May", "June", "July", "August"]:
        print(f"Cash balance at the end of {month}: ${cash_balance_vars[month].X:.2f}")
    for month in months:
        for term in interest_rates:
            print(f"Amount borrowed at the end of {month} for a {term}-month term: ${borrowing_vars[(month, term)].X:.2f}")

    # Print total repayment amount
    print(f"Total repayment amount: ${model.objVal:.2f}")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 12 rows, 13 columns and 36 nonzeros
Model fingerprint: 0x79f2b510
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolved: 8 rows, 11 columns, 31 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.906250e+04   0.000000e+00      0s
       5    1.5053125e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.505312500e+05
Optimal solution found:
Cash balance at the end of May: $25000.00
Cash balance at the end of June: $27500.00
Cash balan

## question(h): Formulate and solve the dual linear program demonstrating that the model you create is, indeed,the correct dual problem of the primal formulation

In [238]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model for the dual
dual_model = gp.Model("SunnyshoreBay_Dual")

# Dual variables for cash balance constraints
y = {month: dual_model.addVar(name=f"y_{month}") for month in ["May", "June", "July", "August"]}

# Dual variables for borrowing limits
w = {month: dual_model.addVar(name=f"w_{month}") for month in ["May", "June", "July"]}

# Dual variable for July cash balance requirement
v = dual_model.addVar(name="v")

# Objective function: Maximize the sum of dual variables times the respective right-hand sides
dual_model.setObjective(
    gp.quicksum(y[month] * min_cash_balance[month] for month in ["May", "June", "July", "August"]) +
    gp.quicksum(w[month] * borrowing_limits[month] for month in ["May", "June", "July"]) +
    v * (0.65 * (min_cash_balance["May"] + min_cash_balance["June"])), GRB.MAXIMIZE)

# Dual constraints derived from primal decision variables
for month in months:
    for term in interest_rates:
        dual_model.addConstr(
            y[month] - (y["August"] if month != "July" else 0) - (w[month] if term != 3 or month != "July" else 0) <= interest_rates[term], 
            f"dual_constraint_{month}_{term}")

# Adding the constraint for v (July cash balance requirement)
dual_model.addConstr(y["July"] - 0.65 * (y["May"] + y["June"]) - v >= 0, "dual_constraint_v")

# Optimize the dual model
dual_model.optimize()

# Print the optimal solutions for the dual variables
#if dual_model.status == GRB.OPTIMAL:
#    print("Optimal solution found for the dual problem:")
    #for var in y.values():
     #   print(f"{var.varName}: {var.X}")
    #for var in w.values():
     #   print(f"{var.varName}: {var.X}")
    #print(f"{v.varName}: {v.X}")
#else:
 #   print("No optimal solution found for the dual problem.")



# Check the model status after optimization
if dual_model.status == GRB.OPTIMAL:
    print("Optimal solution found for the dual problem:")
    # Print the values of the dual variables
elif dual_model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Computing IIS...")
    dual_model.computeIIS()
    dual_model.write("model.ilp")
    print("IIS written to file 'model.ilp'")
elif dual_model.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization was not successful. Model status: {dual_model.status}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 10 rows, 8 columns and 27 nonzeros
Model fingerprint: 0xf6b2ad1a
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
Optimization was not successful. Model status: 4


question(i):  Which formulation, the primal or the dual model, do you think is easier to solve?